# Płytka sieć w TensorFlow

Konstruowanie płytkiej sieci neuronowej klasyfikującej ręcznie pisane litery

### (opcjonalnie) najpierw zainstalujmy tensorflowa...

In [ ]:
!pip install tensorflow matplotlib

### Import zależności (na wstępie ładujemy tensorboarda)

In [ ]:
%load_ext tensorboard
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot as plt

### Wczytajmy obrazy ze zbioru MNIST

In [ ]:
(X_train, y_train), (X_valid, y_valid) = mnist.load_data()

### Sprawdźmy podstawowe cechy zbioru danych

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
y_train[0:12]

In [ ]:
X_valid.shape

In [ ]:
y_valid.shape

### Przejrzyjmy konkretne elementy zbioru danych

In [ ]:
plt.figure(figsize=(5,5))
for k in range(12):
    plt.subplot(3, 4, k+1)
    plt.imshow(X_train[k], cmap='Greys')
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
_ = plt.imshow(X_valid[0], cmap='Greys')

In [ ]:
X_valid[0]

**Zadanie 1a. Co reprezentują liczby w powyższej tablicy?**

tu odpowiedź

In [ ]:
y_valid[0]

### Wykonajmy preprocessing danych

Dokonamy zmiany postaci każdego dwuwymiarowego obrazka w jednowymiarową tablicę o rozmiarze równym liczbie pikseli, jednocześnie zamienimy liczby całkowite z zadania 1 w liczby zmiennoprzecinkowe, by móc je następnie znormalizować (czyli by znajdowały się w przedziale <0,1>.

In [ ]:
X_train = X_train.reshape(60000, 784).astype('float32')
X_valid = X_valid.reshape(10000, 784).astype('float32')

**Zadanie 1b. Znormalizuj wartości w zbiorze uczącym X_train i zbiorze walidującym X_valid**

In [ ]:
# tu odpowiedź

In [ ]:
X_valid[0]

Następnie deklarujemy liczbę możliwych klas i konwertujemy etykiety na kategorie poprzez tzw. one-hot encoding

In [ ]:
n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_valid = to_categorical(y_valid, n_classes)

In [ ]:
y_valid[0]

### Zaprojektujmy architekturę sieci

In [ ]:
model = Sequential()
model.add(Dense(64, activation='sigmoid', input_shape=(784,)))
model.add(Dense(10, activation='softmax'))

Sieć ma 3 warstwy:

    - wejściową o 784 neuronach, po jednym dla każdego piksela obrazu
    
    - ukrytą warstwą gęstą (inaczej mówiąc - w pełni połączoną, w której wszystkie neurony połączone są ze wszystkimi z warstwy wcześniejszej), zbudowaną z 64 neuronów sigmoidalnych
    
    - wyjściową, również gęstą, zbudowaną z 10-neuronowej warstwy softmax, po jednym dla każdej kategorii cyfr (0-9)

In [ ]:
model.summary()

**Zadanie 1c. Wyjaśnij podaną w podsumowaniu modelu liczbę parametrów warstw: 50240 i 650**

tu odpowiedź

### Skompilujmy i skonfigurujmy przygotowany model

In [ ]:
lr = 0.01

In [ ]:
model.compile(loss='mean_squared_error', optimizer=SGD(learning_rate=lr), metrics=['accuracy'])

Sieć będzie się uczyć na bazie średniokwadrartowej funkcji straty, a metodą minimalizacji funkcji kosztu będzie SGD

### Przygotujmy środowisko tensorboard do wizualizacji wyników

In [ ]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # tworzymy lokalizację, w której utworzą się logi
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1) # tensorboard jest dodany do procesu uczenia jako callback

### W końcu nauczmy sieć

Wykorzystujemy dane wejściowe treningowe X_train i powiązane etykiety y_train, uczymy w 200 epokach (200 przejść przez wszystkie dane uczące), na 128-obrazkowych paczkach, skuteczność uczenia sprawdzamy już w trakcie uczenia poprzez podanie zbioru walidującego. Chcemy otrzymać wszystkie informacje w trakcie uczenia, stąd argument verbose otrzymuje wartość 1.

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=200, verbose=1, validation_data=(X_valid, y_valid), callbacks=[tensorboard_callback])

### Dokonajmy ewaluacji modelu na zbiorze walidacyjnym

In [ ]:
model.evaluate(X_valid, y_valid)

### Możesz spróbować pozmieniać rozmiar batcha, learning_rate, liczbę neuronów w ukrytej wartwie gęstej (lub nawet optymalizator) i podzielić się zauważkami czy przyniosło to jakąś poprawę lub pogorszenie działania sieci

tu odpowiedź (opcjonalnie)

### Zwizualizujmy parametry działania sieci w tensorboardzie

In [ ]:
%tensorboard --logdir logs/fit --bind_all

### Sprawdźmy jak dokładnie model działa na pojedynczych przypadkach

w tym celu wyciągamy jedną z próbek (jeden z obrazów) ze zbioru walidacyjnego i wykonujemy na niej metodę prognozującą predict

In [ ]:
valid_0 = X_valid[0].reshape(1, 784)

In [ ]:
model.predict(valid_0)

**Zadanie 2a. Co oznaczają wartości znajdujące się w powyższej tablicy i jaka będzie w związku z tym prognoza co do cyfry na obrazie w tej próbce?**

tu odpowiedź

**Zadanie 2b. Napisz kod sprawdzający - dla każdego rodzaju cyfry oddzielnie - jaką średnią pewność miała sieć, gdy stwierdzała że na obrazku występuje dana cyfra, niezależnie od tego czy poprawnie rozpoznała czy nie (nie musi być cały zbiór, wystarczy 500 pierwszych elementów). Chodzi o to by zobaczyć, która cyfra dla sieci jest najłatwiejsza do rozpoznania**